In [12]:
# URL of the TS viewer
tsvBaseURL='https://proba-v-mep.esa.int/api/timeseries/v1.0/ts/'

# Image ID
imageId = 'PROBAV_L3_S10_TOC_NDVI_333M'

# Point of interest (longitude, latitude)
longitude = 11.0
latitude = 11.0

In [2]:
import requests
import urlparse
import datetime
from datetime import date

def pointTSRequest(coverageId, lon, lat, startDate = None, endDate = None):
    
    tsvURL = urlparse.urljoin(tsvBaseURL, coverageId + '/')
    tsvURL = urlparse.urljoin(tsvURL, 'point')
    
    payload = {
        'lon': str(lon),
        'lat': str(lat)
    }    
    
    if startDate != None:
        payload['startDate'] = startDate.strftime('%Y-%m-%d')
    
    if endDate != None:
        payload['endDate'] = endDate.strftime('%Y-%m-%d')
        
    return requests.get(tsvURL, params=payload)

def pointTS(coverageId, lon, lat, startDate = None, endDate = None):
    
    response = pointTSRequest(coverageId, lon, lat, startDate, endDate)
    timeSeries = response.json()['results']    
    timeSeries = map(lambda x:[datetime.datetime.strptime(x['date'], "%Y-%m-%d").date(),x['result']['average']], timeSeries)
    return timeSeries    
    

In [13]:
# Get the point from the time series viewer
ts = pointTS(imageId, longitude, latitude)

In [4]:
# Install some additional Python libraries
import sys

! pip27 install --user mpld3

The directory '/home/driesj/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/driesj/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [14]:
# Plot the data

%matplotlib inline

import matplotlib.pyplot as plt
import mpld3

fig, ax = plt.subplots()
fig.set_size_inches(12, 4, forward=True)
timeStamps = map(lambda x:x[0], ts)
values = map(lambda x:x[1], ts)

scatter = ax.scatter(timeStamps, values)

labels = ['{0} - {1}'.format(i[1],i[0]) for i in ts]
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
mpld3.plugins.connect(fig, tooltip)

mpld3.display(fig)